In [1]:
pip install wikipedia


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=fa4a100af2411e02398070d24ca8f8e7fe0ddf0ee6a918af3bd0131ae685e2b6
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [7]:
import numpy as np
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from gensim.models import FastText
import spacy
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import wikipedia
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam

In [3]:
search_results = wikipedia.search("football")
search_results = search_results[:8]
doc_content = []
max_content_length = 3000

for title in search_results[:4]:
    page_object = wikipedia.page(title)
    con = page_object.content[:max_content_length]
    doc_content.append(con)

    # Extract and print the content of the current page
    # print("Title:", page_object.title)
    # print("Content:", con)
    # print("\n---\n")

# Combine the content from all pages
text = " ".join(doc_content)




In [4]:
def clean_text(text):
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert text to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Clean the combined text
cleaned_text = clean_text(text)
print(cleaned_text)

football is a family of team sports that involve to varying degrees kicking a ball to score a goal unqualified the word football normally means the form of football that is the most popular where the word is used sports commonly called football include association football known as soccer in australia canada south africa the united states and sometimes in ireland and new zealand australian rules football gaelic football gridiron football specifically american football arena football or canadian football international rules football rugby league football and rugby union football these various forms of football share to varying degrees common origins and are known as football codes there are a number of references to traditional ancient or prehistoric ball games played in many different parts of the world contemporary codes of football can be traced back to the codification of these games at english public schools during the th century itself an outgrowth of medieval football the expansi

In [5]:
#character mappings
chars = sorted(list(set(cleaned_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))


In [6]:
# Create one-hot encoded sequences
max_length = len(cleaned_text)
num_chars = len(chars)

X = []
y = []

seq_length = 20

for i in range(0, max_length - seq_length, 1):
    seq_in = cleaned_text[i:i + seq_length]
    seq_out = cleaned_text[i + seq_length]
    X.append([char_to_int[char] for char in seq_in])
    y.append(char_to_int[seq_out])

# Reshape X to be [samples(#samples), time steps(seq_length), features(#unique char)]
X_modified = np.reshape(X, (len(X), seq_length, 1))

# normalize
X_modified = X_modified / float(num_chars)

# one-hot encode the output variable
ys = tf.keras.utils.to_categorical(y, num_classes=num_chars)



#using LSTM model

In [ ]:


# Define the model
model = Sequential()

model.add(LSTM(256, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(ys.shape[1], activation='softmax'))

# Compile the model
opt = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt)

print(model.summary())



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 20, 256)           264192    
                                                                 
 lstm_3 (LSTM)               (None, 128)               197120    
                                                                 
 dense_1 (Dense)             (None, 27)                3483      
                                                                 
Total params: 464795 (1.77 MB)
Trainable params: 464795 (1.77 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# Train the model
model.fit(X_modified, ys, epochs=40, batch_size=128)


Epoch 1/40
91/91 [==============================] - 25s 223ms/step - loss: 2.9042
Epoch 2/40
91/91 [==============================] - 21s 236ms/step - loss: 2.8654
Epoch 3/40
91/91 [==============================] - 21s 226ms/step - loss: 2.8644
Epoch 4/40
91/91 [==============================] - 21s 229ms/step - loss: 2.8569
Epoch 5/40
91/91 [==============================] - 22s 238ms/step - loss: 2.8323
Epoch 6/40
91/91 [==============================] - 20s 221ms/step - loss: 2.8146
Epoch 7/40
91/91 [==============================] - 22s 241ms/step - loss: 2.7840
Epoch 8/40
91/91 [==============================] - 20s 221ms/step - loss: 2.7410
Epoch 9/40
91/91 [==============================] - 22s 242ms/step - loss: 2.6846
Epoch 10/40
91/91 [==============================] - 22s 245ms/step - loss: 2.6232
Epoch 11/40
91/91 [==============================] - 21s 235ms/step - loss: 2.5734
Epoch 12/40
91/91 [==============================] - 21s 227ms/step - loss: 2.5109
Epoch 13/40
9

In [ ]:
# Train the model with additional epochs
model.fit(X_modified, ys, epochs=40, batch_size=128)

Epoch 1/40
91/91 [==============================] - 31s 343ms/step - loss: 0.2864
Epoch 2/40
91/91 [==============================] - 24s 264ms/step - loss: 0.2245
Epoch 3/40
91/91 [==============================] - 28s 310ms/step - loss: 0.1774
Epoch 4/40
91/91 [==============================] - 23s 257ms/step - loss: 0.1500
Epoch 5/40
91/91 [==============================] - 22s 242ms/step - loss: 0.1285
Epoch 6/40
91/91 [==============================] - 23s 252ms/step - loss: 0.1159
Epoch 7/40
91/91 [==============================] - 23s 256ms/step - loss: 0.1449
Epoch 8/40
91/91 [==============================] - 24s 261ms/step - loss: 0.1052
Epoch 9/40
91/91 [==============================] - 21s 228ms/step - loss: 0.0894
Epoch 10/40
91/91 [==============================] - 23s 255ms/step - loss: 0.0774
Epoch 11/40
91/91 [==============================] - 21s 230ms/step - loss: 0.0693
Epoch 12/40
91/91 [==============================] - 23s 252ms/step - loss: 0.1451
Epoch 13/40
9

In [16]:

def generate_text(model, seq_length, char_to_int, int_to_char, num_chars, seed_text, num_chars_to_generate):

    generated_text = seed_text

    # Generate characters
    for _ in range(num_chars_to_generate):
        # Preprocess the seed text
        if len(seed_text) < seq_length:
            x = np.reshape([char_to_int[char] for char in seed_text], (1, len(seed_text), 1))
        else:
            x = np.reshape([char_to_int[char] for char in seed_text[-seq_length:]], (1, seq_length, 1))
        x = x / float(num_chars)

        # Predict the next character probabilities
        predictions = model.predict(x, verbose=0)

        # Convert predictions to character index
        predicted_char_index = np.argmax(predictions)

        # Convert character index to actual character
        predicted_char = int_to_char[predicted_char_index]

        # Append the predicted character to the generated text
        generated_text += predicted_char

        # Update the seed text by removing the first character and adding the predicted character
        seed_text = seed_text[1:] + predicted_char

    return generated_text

In [ ]:
# generate text
seed_text = "the football is sport which is "
num_chars_to_generate = 300
generated_text = generate_text(model, seq_length, char_to_int, int_to_char, num_chars, seed_text, num_chars_to_generate)

# Print the generated text
print(generated_text)

the football is sport which is the mfl canhderen somng gootnnt terccnd pt mivedat thck frrmrwing the conccusoons of gue with ameiiseshens loveddacn there creakice nd kasco aachning e fentiapaenglint she spilts in uhe world with the individual games accounting for many of the most watched television programs in american history an


#using simple RNN

In [12]:
# Define the model
model = Sequential()

model.add(SimpleRNN(256, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(SimpleRNN(128, return_sequences=True))
model.add(SimpleRNN(64))

model.add(Dense(ys.shape[1], activation='softmax'))

# Compile the model
opt = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt)

print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_10 (SimpleRNN)   (None, 20, 256)           66048     
                                                                 
 simple_rnn_11 (SimpleRNN)   (None, 20, 128)           49280     
                                                                 
 simple_rnn_12 (SimpleRNN)   (None, 64)                12352     
                                                                 
 dense_2 (Dense)             (None, 27)                1755      
                                                                 
Total params: 129435 (505.61 KB)
Trainable params: 129435 (505.61 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [14]:
model.fit(X_modified, ys, epochs=80, batch_size=128)


Epoch 1/80
91/91 [==============================] - 8s 84ms/step - loss: 2.8599
Epoch 2/80
91/91 [==============================] - 7s 76ms/step - loss: 2.8491
Epoch 3/80
91/91 [==============================] - 7s 76ms/step - loss: 2.8157
Epoch 4/80
91/91 [==============================] - 7s 78ms/step - loss: 2.7640
Epoch 5/80
91/91 [==============================] - 7s 75ms/step - loss: 2.7198
Epoch 6/80
91/91 [==============================] - 7s 79ms/step - loss: 2.6699
Epoch 7/80
91/91 [==============================] - 7s 73ms/step - loss: 2.6066
Epoch 8/80
91/91 [==============================] - 7s 80ms/step - loss: 2.5493
Epoch 9/80
91/91 [==============================] - 6s 71ms/step - loss: 2.4958
Epoch 10/80
91/91 [==============================] - 7s 81ms/step - loss: 2.4494
Epoch 11/80
91/91 [==============================] - 7s 73ms/step - loss: 2.4053
Epoch 12/80
91/91 [==============================] - 7s 81ms/step - loss: 2.3710
Epoch 13/80
91/91 [==================

In [18]:
model.fit(X_modified, ys, epochs=40, batch_size=128)


Epoch 1/40
91/91 [==============================] - 7s 81ms/step - loss: 0.3918
Epoch 2/40
91/91 [==============================] - 7s 72ms/step - loss: 0.3793
Epoch 3/40
91/91 [==============================] - 7s 81ms/step - loss: 0.3546
Epoch 4/40
91/91 [==============================] - 6s 71ms/step - loss: 0.3579
Epoch 5/40
91/91 [==============================] - 7s 79ms/step - loss: 0.3468
Epoch 6/40
91/91 [==============================] - 7s 73ms/step - loss: 0.3694
Epoch 7/40
91/91 [==============================] - 7s 79ms/step - loss: 0.3437
Epoch 8/40
91/91 [==============================] - 7s 75ms/step - loss: 0.3515
Epoch 9/40
91/91 [==============================] - 7s 80ms/step - loss: 0.3624
Epoch 10/40
91/91 [==============================] - 7s 74ms/step - loss: 0.3154
Epoch 11/40
91/91 [==============================] - 7s 80ms/step - loss: 0.2974
Epoch 12/40
91/91 [==============================] - 7s 74ms/step - loss: 0.3137
Epoch 13/40
91/91 [==================

In [20]:
# generate text
seed_text = "the football is sport which is "
num_chars_to_generate = 300
generated_text = generate_text(model, seq_length, char_to_int, int_to_char, num_chars, seed_text, num_chars_to_generate)

# Print the generated text
print(generated_text)

the football is sport which is eolleger i nesel a moges of puhes and comticered coelest ofaauedr frous lacn gne amnowedt mn the eirst lary coaruns erouna poeters of sugogat mnly the orpoariyhds ans  nllndil the ball a ciriouel iortball astociatien watde to ale koowe t thar fial gootball league nfl has the highest average attendan
